## Get Links and Images

* Script goes through the watch listing pages such as [here](https://www.prestigetime.com/luxury-watches-for-men.html).
* Download all of the watch pictures
* Get links for all individual watch pages (so that I can go back later and get more watch attributes).
* Save basic watch attributes to a csv.

Note: Due to the structure of the PrestigeTime website, code is run seperately for men's and womens watches.

In [3]:
# General Libraries
import os
import re
import time
import csv

# Analysis Libraries
import numpy as np
import pandas as pd

# Scraping Libraries
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent

Randomize Me to Prevent Getting Blocked

In [1]:
def random_soup(url):
    us = UserAgent()
    user_agent = {'User-Agent':us.random}
    response = requests.get(url,headers = user_agent)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    return soup

__Loop through all of the pages__

In [ ]:
#Get all Watch Links (Not Previously Saved)
#num_pages = 158 #mens
num_pages = 127 #womens
watch_list = []
for ix in range(1,num_pages+1):
    time.sleep(1+np.random.uniform(0,2)) 
    #listing_soup = random_soup('https://www.prestigetime.com/luxury-watches-for-men.html&page='+str(ix))
    listing_soup = random_soup('https://www.prestigetime.com/luxury-watches-for-women.html&page='+str(ix))
    listings = listing_soup.find_all('div',class_='thumbnail thumbnail-center')

    for listing in listings:

        watch_dict = dict()
        #URL
        watch_dict['url'] = listing.find('a')['href']

        #Image URL
        watch_dict['image_url'] = listing.find('img')['src']

        #Brand
        watch_dict['brand'] = listing.find('strong').text.strip()

        # Model Name 
        watch_dict['model_name'] = listing.find('span',id=re.compile("series-")).text.strip()

        # Model Number
        watch_dict['model_num'] = listing.find('span',id=re.compile("model_no")).text.strip()

        #Price
        price = listing.find('div',class_="caption-bottom").text
        price = price.split(':')[1]
        price = price.replace(',','')
        price = re.findall(r"(\d+)\.(\d+)", price)
        try:
            price = float(price[0][0] +'.'+ price[0][1])
        except:
            IndexError
            price = np.nan

        watch_dict['price'] = price 


        # Image Name
        image_name = watch_dict['brand'] + watch_dict['model_name'] + watch_dict['model_num']
        image_name = re.sub('[^0-9a-zA-Z]+', '', image_name)
        watch_dict['image_name'] = image_name

        watch_list.append(watch_dict)

        if image_name+'.jpg' not in os.listdir('prestige_time_pics/'):
            # Download Image
            us = UserAgent()
            user_agent = {'User-Agent':us.random}

            time.sleep(1+np.random.uniform(0,2)) 
            response = requests.get(watch_dict['image_url'],headers = user_agent)
            #f = open('../prestige_time_pics_mens/'+image_name+'.jpg','wb')
            f = open('../prestige_time_pics_womens/'+image_name+'.jpg','wb')
            f.write(requests.get(watch_dict['image_url']).content)
            f.close()

    print(ix)

1
2
3
4
5
6


__Save Summaries to CSV__

In [ ]:
watch_DF = pd.DataFrame(watch_list)
watch_DF.to_csv('watch_page_list_womens.csv',index=False)
#watch_DF.to_csv('watch_page_list_mens.csv',index=False)